In [1]:
import sqlite3
import time
import statistics

In [2]:
db_path = 'Dirigo.sqlite'

query = '''
WITH IdentifiedTruck AS (
  SELECT Object_id AS TruckID
  FROM OCEL_Truck
  WHERE LPT = '926VFT'
),
EventTime AS (
SELECT Timestamp
FROM OCEL_O2O
WHERE Source_object_id = (SELECT TruckID FROM IdentifiedTruck)
AND O2O_qualifier LIKE 'dropped%'
AND Target_object_id = 'Pcp18'
),
EventID AS (
SELECT Event_id
FROM OCEL_Events
WHERE Timestamp = (SELECT Timestamp FROM EventTime)
)
SELECT E2O_qualifier
FROM OCEL_E2O
WHERE Event_id = (SELECT Event_id FROM EventID)
AND Object_id = (SELECT TruckID FROM IdentifiedTRuck);
'''
repeat = 1000

conn = sqlite3.connect(db_path)
cursor = conn.cursor()


durations = []

for i in range(repeat):
    start_time = time.time()
    cursor.execute(query)
    cursor.fetchall()  
    end_time = time.time()
    
    duration = (end_time - start_time) * 1000  
    durations.append(duration)
    # print(f"Run {i+1}: {duration:.4f} ms")


avg = statistics.mean(durations)
std_dev = statistics.stdev(durations)

print("\n=== Execution Statistics (Dirigo) ===")
print(f"Average execution time: {avg:.4f} ms")
print(f"Standard deviation: {std_dev:.4f} ms")


cursor.close()
conn.close()


=== Execution Statistics (Dirigo) ===
Average execution time: 0.0990 ms
Standard deviation: 0.0400 ms


In [3]:
db_path = 'ACEL.sqlite'

query = """
WITH IdentifiedTruck AS (
  SELECT ObjectId AS TruckID
  FROM ACEL_Objects
  WHERE LPT = '926VFT'
),
TruckEvents AS (
SELECT e.*
FROM ACEL_Events e
WHERE EXISTS (
  SELECT 1
  FROM json_each(REPLACE(e.Objects, '''', '"'))
  WHERE json_each.value = (SELECT TruckID FROM IdentifiedTruck))
 ),
RelationsE2O AS (
SELECT Relations
FROM TruckEvents, json_each(REPLACE(ObjectChanges, '''', '"')) AS jc
WHERE json_extract(jc.value, '$.Attribute') = 'Pickup Plan ID'
AND json_extract(jc.value, '$.NewValue') = 'Pcp18'
),
SELECTEDEvents AS (
SELECT * 
FROM ACEL_Events 
WHERE Relations = (SELECT Relations FROM RelationsE2O)
)
SELECT json_extract(jc.value, '$.NewValue') 
FROM SELECTEDEvents, json_each(REPLACE(ObjectChanges, '''', '"')) AS jc
WHERE json_extract(jc.value, '$.Attribute') = 'lifecycle'
AND Activity = 'Evaluate the Truck Exit';
"""
repeat = 1000

conn = sqlite3.connect(db_path)
cursor = conn.cursor()


durations = []

for i in range(repeat):
    start_time = time.time()
    cursor.execute(query)
    cursor.fetchall()  
    end_time = time.time()
    
    duration = (end_time - start_time) * 1000  
    durations.append(duration)
    # print(f"Run {i+1}: {duration:.4f} ms")


avg = statistics.mean(durations)
std_dev = statistics.stdev(durations)

print("\n=== Execution Statistics (Dirigo) ===")
print(f"Average execution time: {avg:.4f} ms")
print(f"Standard deviation: {std_dev:.4f} ms")


cursor.close()
conn.close()


=== Execution Statistics (Dirigo) ===
Average execution time: 0.1507 ms
Standard deviation: 0.0777 ms
